In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import string
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from assnat.main import complete_preproc

2024-08-22 12:29:27.828864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('data/leg16.csv')
data = pd.read_csv('data/leg16.csv')

In [3]:
df

,Unnamed: 0,Nom Orateur,ID Orateur,Texte,Thème Séance,ID Séance,Date Séance,ID Session,grammaire,famille
0,0,Mme la présidente,719874,La séance est ouverte.,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,OUV_SEAN_2_1,Centre
1,1,Mme la présidente,719874,L’ordre du jour appelle la discussion du proje...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,ODJ_APPEL_DISCUSSION,Centre
2,2,Mme la présidente,719874,La parole est à Mme la ministre de la transiti...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre
3,3,Mme Agnès Pannier-Runacher,759832,"Après la promulgation, vendredi, de la loi rel...",Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit
4,4,M. Guillaume Kasbarian,719372,Elle a raison !,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,INTERRUPTION_1_10,Centre
...,...,...,...,...,...,...,...,...,...,...
312092,312092,M. Gérard Leseul (SOC),774958,Je remercie le groupe GDR-NUPES d’avoir mis ce...,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE,Gauche
312093,312093,Mme la présidente,720908,La parole est à M. le ministre.,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit
312094,312094,M. Stanislas Guerini,721498,Je le dis sous le contrôle de Mme la président...,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre
312095,312095,Mme la présidente,720908,Le débat est clos.,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit


In [4]:
df['Nom Orateur'].value_counts(dropna = False)

Mme la présidente                                    81699
M. le président                                      10729
M. Éric Dupond-Moretti                                3333
M. Gabriel Attal                                      3002
M. Sylvain Maillard                                   2903
                                                     ...  
M. Christophe Bex et M. Laurent Alexandre                1
Mme Maud Bregeon et M. Sylvain Maillard                  1
Mme Elsa Faucillon (GDR)                                 1
M. Philippe Ballard                                      1
M. Joël Giraud et plusieurs députés du groupe Dem        1
Name: Nom Orateur, Length: 2255, dtype: int64

In [5]:
def drop_certain_names(df,names):
    df = df[~df['Nom Orateur'].isin(names)]
    return df

drop_certain_names(df, ['Mme la présidente','M. le président'])


,Unnamed: 0,Nom Orateur,ID Orateur,Texte,Thème Séance,ID Séance,Date Séance,ID Session,grammaire,famille
3,3,Mme Agnès Pannier-Runacher,759832,"Après la promulgation, vendredi, de la loi rel...",Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit
4,4,M. Guillaume Kasbarian,719372,Elle a raison !,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,INTERRUPTION_1_10,Centre
5,5,Mme Agnès Pannier-Runacher,759832,Comment se dire écologiste lorsqu’on ne vote p...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit
6,6,M. Pierre Cazeneuve,795864,Eh oui !,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,INTERRUPTION_1_10,Centre
7,7,Mme Agnès Pannier-Runacher,759832,…lorsqu’on vante le modèle allemand qui repose...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit
...,...,...,...,...,...,...,...,...,...,...
312086,312086,M. Stanislas Guerini,721498,Il n’est pas un jour où je ne parle pas des se...,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre
312088,312088,M. André Chassaigne (GDR-NUPES),267306,"Monsieur le ministre, il faudrait des heures, ...",Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE,Gauche
312090,312090,M. Stanislas Guerini,721498,"Premièrement, est-il légitime pour l’État de s...",Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre
312092,312092,M. Gérard Leseul (SOC),774958,Je remercie le groupe GDR-NUPES d’avoir mis ce...,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE,Gauche


In [6]:
df['Nom Orateur'].value_counts(dropna = False)

Mme la présidente                                    81699
M. le président                                      10729
M. Éric Dupond-Moretti                                3333
M. Gabriel Attal                                      3002
M. Sylvain Maillard                                   2903
                                                     ...  
M. Christophe Bex et M. Laurent Alexandre                1
Mme Maud Bregeon et M. Sylvain Maillard                  1
Mme Elsa Faucillon (GDR)                                 1
M. Philippe Ballard                                      1
M. Joël Giraud et plusieurs députés du groupe Dem        1
Name: Nom Orateur, Length: 2255, dtype: int64

In [7]:
def drop_na(df, column):
    df = df.dropna(subset = column)
    return df

In [8]:
df = drop_na(data, ['Texte','famille'])

In [9]:
df.isnull().sum()

Unnamed: 0       0
Nom Orateur     31
ID Orateur       0
Texte            0
Thème Séance     0
ID Séance        0
Date Séance      0
ID Session       0
grammaire        0
famille          0
dtype: int64

In [10]:
df.shape

(311624, 10)

In [11]:
df.isnull().sum()

Unnamed: 0       0
Nom Orateur     31
ID Orateur       0
Texte            0
Thème Séance     0
ID Séance        0
Date Séance      0
ID Session       0
grammaire        0
famille          0
dtype: int64

In [12]:
def remove_short_sentences(df, n):
    def word_count(sentence):
        return len(sentence.split())
    df = df[df['Texte'].apply(word_count) >= n]
    return df

In [13]:
def apply_preprocessing(df, punct_opt = True):
    def preprocessing(sentence, punct_option = True):
        # Removing whitespaces
        sentence = sentence.strip()
        # Lowercasing
        sentence = sentence.lower()
        # Removing punctuation
        if punct_option == True:
            for punctuation in string.punctuation.replace('?','').replace('!',''):
                sentence = sentence.replace(punctuation, '')
        else:
            for punctuation in string.punctuation:
                sentence = sentence.replace(punctuation, '')
        return sentence
    df['Texte'] = df['Texte'].apply(preprocessing)
    df['Texte'] = df['Texte'].apply(text_to_word_sequence,filters=string.punctuation.replace('?','').replace('!',''), lower=True, split=' ')
    return df

In [14]:
df_test = df.head(2)
df_test

,Unnamed: 0,Nom Orateur,ID Orateur,Texte,Thème Séance,ID Séance,Date Séance,ID Session,grammaire,famille
0,0,Mme la présidente,719874,La séance est ouverte.,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,OUV_SEAN_2_1,Centre
1,1,Mme la présidente,719874,L’ordre du jour appelle la discussion du proje...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,ODJ_APPEL_DISCUSSION,Centre


In [15]:
apply_preprocessing(df_test)

/var/folders/mp/99nc3g2s3t192ft55tyrlzdh0000gn/T/ipykernel_71763/169491108.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Texte'] = df['Texte'].apply(preprocessing)
/var/folders/mp/99nc3g2s3t192ft55tyrlzdh0000gn/T/ipykernel_71763/169491108.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Texte'] = df['Texte'].apply(text_to_word_sequence,filters=string.punctuation.replace('?','').replace('!',''), lower=True, split=' ')


,Unnamed: 0,Nom Orateur,ID Orateur,Texte,Thème Séance,ID Séance,Date Séance,ID Session,grammaire,famille
0,0,Mme la présidente,719874,"[la, séance, est, ouverte]",Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,OUV_SEAN_2_1,Centre
1,1,Mme la présidente,719874,"[l’ordre, du, jour, appelle, la, discussion, d...",Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,ODJ_APPEL_DISCUSSION,Centre


In [16]:
test = text_to_word_sequence("I like cheese ?", filters=string.punctuation.replace('?','').replace('!',''), lower=True, split=' ')
test

['i', 'like', 'cheese', '?']

In [17]:
df_preproc = complete_preproc(df, na_col = 'Texte', drop_names = ['Mme la présidente', 'M. le président'], min_words = 6, punct_opt= True)

Upload achieved
   Unnamed: 0                 Nom Orateur  ID Orateur  \
0           0           Mme la présidente      719874   
1           1           Mme la présidente      719874   
2           2           Mme la présidente      719874   
3           3  Mme Agnès Pannier-Runacher      759832   
4           4      M. Guillaume Kasbarian      719372   

                                               Texte  \
0                             La séance est ouverte.   
1  L’ordre du jour appelle la discussion du proje...   
2  La parole est à Mme la ministre de la transiti...   
3  Après la promulgation, vendredi, de la loi rel...   
4                                    Elle a raison !   

                                        Thème Séance               ID Séance  \
0  Accélération des procédures liées à la constru...  RUANR5L16S2023IDS26877   
1  Accélération des procédures liées à la constru...  RUANR5L16S2023IDS26877   
2  Accélération des procédures liées à la constru...  RUANR5L16S

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df_preproc